# The Hangman Game

Creating Vocabulary

In [1]:
char2idx = {'<PAD>' : 0}
idx2char = ['<PAD>']

for i in range(26):
    char2idx[chr(i + ord('a'))] = i + 1
    idx2char.append(chr(ord('a') + i))

char2idx['_'] = 27
idx2char.append('_')

Importing required packages

In [2]:
import tensorflow as tf
import numpy as np
tf.keras.backend.clear_session()

2023-07-25 06:53:43.150522: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 06:53:46.686191: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from keras import Input, Model
from keras.layers import LSTM, Bidirectional, Embedding, Dense, TimeDistributed, Attention, Concatenate, Masking, Dropout, GlobalMaxPooling1D

Creating Model

In [4]:
LATENT_DIM = 256

input1 = Input(shape=(None,), name = "Input_Word")

embedding_layer = Embedding(input_dim = 28, output_dim = 40, mask_zero = True, name = "Embedding_Layer")
emb_input = embedding_layer(input1)

bilstm_layer = Bidirectional(LSTM(LATENT_DIM, return_sequences = True, name = "Bidirection_LSTM_Layer", dropout = 0.1))
lstm_output = bilstm_layer(emb_input)

pool_layer = GlobalMaxPooling1D(name = "MaxPooling_Layer")
lstm_output = pool_layer(lstm_output)

input2 = Input(shape = (26,), name = "Correct_Guesses")
input3 = Input(shape = (26,), name = "Incorrect_Guesses")


dense_layer = Dense(128, activation = 'relu', name = "Dense_Layer")
dense_layer1 = Dense(128, activation = 'relu', name = "Dense_Layer1")
correct_guess_output = dense_layer(input2)
correct_guess_output = Dropout(0.1, seed = 218)(correct_guess_output)
incorrect_guess_output = dense_layer1(input3)
incorrect_guess_output = Dropout(0.1, seed = 218)(incorrect_guess_output)


concat_layer = Concatenate(name = "Concatenate_Layer")
guess_output = concat_layer([correct_guess_output, incorrect_guess_output])

dense_layer2 = Dense(256, activation = 'relu', name = "Dense_Layer2")
guess_output = dense_layer2(guess_output)
guess_output = Dropout(0.2, seed = 218)(guess_output)


concat_output = concat_layer([lstm_output, guess_output])

dense_layera = Dense(512, activation='relu', name = "Dense_LayerA")
dense_layerb = Dense(256, activation='relu', name = "Dense_LayerB")
dense_layerc = Dense(26, activation='sigmoid', name = "Dense_LayerC")
output = dense_layera(concat_output)
output = Dropout(0.3, seed = 218)(output)
output = dense_layerb(output)
output = Dropout(0.2, seed = 218)(output)
output = dense_layerc(output)

model = Model([input1, input2, input3], output)

2023-07-25 06:54:03.127950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14763 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0001:00:00.0, compute capability: 7.5


Loading Trained model weights

In [6]:
model.load_weights('model_weights.h5')